## Google Photos API

This code samples should
- connect to Google Photos API
- list all stored Photos in my Google Photos Acc
- download specific photos

In [1]:
import pandas as pd

# import customized modules
import sys
import os
sys.path.insert(0,r".\helper_scripts")

from google_photos_api import GooglePhotosApi

In [2]:
current_directory = os.getcwd()
current_directory

'C:\\Repos\\google-photos-api\\GooglePhotosApi'

In [3]:
# create serice
google_photos_api = GooglePhotosApi()
service = google_photos_api.create_service()

photoslibrary service created successfully


In [4]:
import os
cwd = os.getcwd()
cwd

'C:\\Repos\\google-photos-api'

In [13]:
# list all media items in Google Photos API
media_items = service.mediaItems().search().execute()
df = pd.DataFrame(media_items)

In [27]:
media_items['mediaItems'][0]['id']

'AIi7vOhwO1J_kHWOYvzwnnq0qI6RRYhV4uMjDNVePv-Cgaev_v_BYNwiSVrp8T_7lFe6iPdT-5QkJF6RWWJxSJphygCIioa06g'

In [29]:
print(dir(service.mediaItems().get))

['__call__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__func__', '__ge__', '__get__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__self__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


In [32]:
help(service.mediaItems().get)

Help on method method in module googleapiclient.discovery:

method(**kwargs) method of googleapiclient.discovery.Resource instance
    Returns the media item for the specified media item identifier.
    
    Args:
      mediaItemId: string, Required. Identifier of the media item to be requested. (required)
      x__xgafv: string, V1 error format.
        Allowed values
          1 - v1 error format
          2 - v2 error format
    
    Returns:
      An object of the form:
    
        { # Representation of a media item (such as a photo or video) in Google Photos.
      "baseUrl": "A String", # A URL to the media item's bytes. This shouldn't be used as is. Parameters should be appended to this URL before use. See the [developer documentation](https://developers.google.com/photos/library/guides/access-media-items#base-urls) for a complete list of supported parameters. For example, `'=w2048-h1024'` will set the dimensions of a media item of type photo to have a width of 2048 px and heig

In [60]:
import requests
url = media_items['mediaItems'][0]['productUrl']
respons = requests.get(url)
file_name = media_items['mediaItems'][0]['filename']
with open(os.path.join(destination_folder, file_name), 'wb') as f:
    f.write(response.content)
    f.close()

NameError: name 'destination_folder' is not defined

In [61]:
import boto3

client = boto3.client('s3', region_name='us-west-2')

client.upload_file('images/image_0.jpg', 'mybucket', 'image_0.jpg')

ModuleNotFoundError: No module named 'boto3'

In [63]:
dynamodb = boto3.resource('dynamodb', endpoint_url="http://localhost:8000")
dynamodb

NameError: name 'boto3' is not defined

In [37]:
dir(service.mediaItems().get(mediaItemId=media_items['mediaItems'][0]['id']))

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_in_error_state',
 '_process_response',
 '_rand',
 '_sleep',
 'add_response_callback',
 'body',
 'body_size',
 'execute',
 'from_json',
 'headers',
 'http',
 'method',
 'methodId',
 'next_chunk',
 'null_postproc',
 'postproc',
 'response_callbacks',
 'resumable',
 'resumable_progress',
 'resumable_uri',
 'to_json',
 'uri']